#Use Census Data API instead to make the income dataset- this is to prevent multiple zip codes
#and to make a cleaner/more transparent merge

In [1]:
# Dependencies
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
#from us import states


# Census API Key
c = Census(census_key, year=2017)

In [2]:
# Run Census Data API to retrieve data on all zip codes (2017 ACS5 Census)
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Clean up columns
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

#Calculate poverty rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]

# Check number of records and dataframe
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,00601,17599.0,38.9,11757.0,7041.0,11282.0,64.105915
1,00602,39209.0,40.9,16190.0,8978.0,20428.0,52.100283
2,00603,50135.0,40.4,16645.0,10897.0,25176.0,50.216416
3,00606,6304.0,42.8,13387.0,5960.0,4092.0,64.911168
4,00610,27590.0,41.4,18741.0,9266.0,12553.0,45.498369


In [3]:
# File to Load- obtain Walmart csv
file_to_load = "Walmart.csv"


In [4]:
#Make into dataframe
walmart_pd = pd.read_csv(file_to_load, encoding="utf-8")
walmart_pd.head()

,Store Number,Store Name
0,4,"Walmart Supercenter, 2901 Highway 412 E, Siloa..."
1,8,"Walmart Supercenter, 1621 North Business 9, Mo..."
2,5,"Walmart Supercenter, 1155 Hwy 65 North, Conway..."
3,7,"Walmart Supercenter, 9053 Highway 107, Sherwoo..."
4,3,"Walmart Supercenter, 30983 Highway 441 S, Comm..."


In [5]:
#Split Store Name into address fields
walmart_pd['Store Name'] = walmart_pd['Store Name'].str.replace(' - Walmart.com','')
walmart_pd[['Store Type', 'Address', 'City', 'State Zip']] = walmart_pd['Store Name'].str.split(",", n = 3, expand = True)
walmart_pd[['B','State','Zipcode']] = walmart_pd['State Zip'].str.split(" ",n=2, expand=True)
walmart_pd = walmart_pd.drop(columns=['Store Name','State Zip','B'])

walmart_pd.head()

,Store Number,Store Type,Address,City,State,Zipcode
0,4,Walmart Supercenter,2901 Highway 412 E,Siloam Springs,AR,72761
1,8,Walmart Supercenter,1621 North Business 9,Morrilton,AR,72110
2,5,Walmart Supercenter,1155 Hwy 65 North,Conway,AR,72032
3,7,Walmart Supercenter,9053 Highway 107,Sherwood,AR,72120
4,3,Walmart Supercenter,30983 Highway 441 S,Commerce,GA,30529


In [6]:
#Strip any trailing and/or leading zeros

walmart_pd[['Zipcode']] = walmart_pd['Zipcode'].str.strip()
walmart_pd.head()

,Store Number,Store Type,Address,City,State,Zipcode
0,4,Walmart Supercenter,2901 Highway 412 E,Siloam Springs,AR,72761
1,8,Walmart Supercenter,1621 North Business 9,Morrilton,AR,72110
2,5,Walmart Supercenter,1155 Hwy 65 North,Conway,AR,72032
3,7,Walmart Supercenter,9053 Highway 107,Sherwood,AR,72120
4,3,Walmart Supercenter,30983 Highway 441 S,Commerce,GA,30529


In [7]:
#Take the last 5 of the zip code variable
walmart_pd['Zipcode'] = walmart_pd['Zipcode'].str[-5:]
#This variable from the census data is character and does not need to be converted to numeric for merging
#This is contrary to the previous income csv file that we were working with

In [8]:
#Merge
walinc_table_pd = pd.merge(walmart_pd, census_pd, on="Zipcode", how="left")

#Check new dataframe
walinc_table_pd.head()

,Store Number,Store Type,Address,City,State,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,4,Walmart Supercenter,2901 Highway 412 E,Siloam Springs,AR,72761,22314.0,31.5,48064.0,21146.0,3887.0,17.419557
1,8,Walmart Supercenter,1621 North Business 9,Morrilton,AR,72110,10913.0,40.0,37808.0,22314.0,2302.0,21.094108
2,5,Walmart Supercenter,1155 Hwy 65 North,Conway,AR,72032,32508.0,34.1,44686.0,22699.0,5311.0,16.337517
3,7,Walmart Supercenter,9053 Highway 107,Sherwood,AR,72120,33161.0,38.7,63575.0,30459.0,3654.0,11.018968
4,3,Walmart Supercenter,30983 Highway 441 S,Commerce,GA,30529,10766.0,36.8,44854.0,21262.0,1840.0,17.090842


In [9]:
#Check the length of the new dataframe
print(len(walinc_table_pd))

4977


In [10]:
#Check the length of the new dataframe
print(len(walmart_pd))

4977


In [11]:
#They match

In [12]:
#Save to CSV
census_pd.to_csv('census_k.csv')
walinc_table_pd.to_csv('combineddata.csv')